In [90]:
import os
import sys
import numpy as np
import pandas as pd
import pandas_datareader as pdr

# Hack to ensure the notebook can load local modules by appending the parent directory to the path
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))
from alphasim.backtest import backtest

In [91]:
price_df = pdr.get_data_yahoo(['VTI', 'TLT'])
price_df = price_df['Adj Close']

display(price_df)

Symbols,VTI,TLT
Date,,
2017-10-09,120.264023,111.911880
2017-10-10,120.548782,112.092407
2017-10-11,120.714127,112.309074
2017-10-12,120.567146,112.787476
2017-10-13,120.658981,113.599854
...,...,...
2022-09-29,181.940002,103.542809
2022-09-30,179.470001,102.206001
2022-10-03,184.029999,103.830002


In [92]:
weight_df = price_df.copy()
weight_df['VTI'] = 0.6
weight_df['TLT'] = 0.4

display(weight_df)

Symbols,VTI,TLT
Date,,
2017-10-09,0.6,0.4
2017-10-10,0.6,0.4
2017-10-11,0.6,0.4
2017-10-12,0.6,0.4
2017-10-13,0.6,0.4
...,...,...
2022-09-29,0.6,0.4
2022-09-30,0.6,0.4
2022-10-03,0.6,0.4


In [93]:
def min_commission(trade_price, trade_size):
    return 10

In [94]:
max_lev = 1
vola_target = 0.10
trade_buffer = 0.05
initial_cap = 10000.0
result = backtest(price_df, weight_df, min_commission, trade_buffer, initial_cap)

In [95]:
assert(price_df.shape == weight_df.shape)

price_df['cash'] = 1
weight_df['cash'] = max_lev - weight_df.sum(axis=1)

pf_df = pd.DataFrame(index=price_df.index, columns=weight_df.columns)
pf_df[:] = 0.0
pf_df.iloc[0,-1] = initial_cap

pf_df.iloc[0,0] = 100
pf_df.iloc[0,1] = 10

pf_df.iloc[1,0] = 30
pf_df.iloc[1,1] = 8

pnl_df = pf_df.copy()


'''
current_weight = (size * price) / nav
target_weight = weight
delta_weight = target_weight - current_weight
do_trade = delta_weight > trade_buffer

In fixed_pct_comm scheme:
trade_weight = target_weight - trade_buffer

In min_comm scheme:
trade_weight = target_weight

trade_value = trade_weight * nav
trade_size = trade_value / price
'''

display(price_df, weight_df, pf_df, pnl_df)

Symbols,VTI,TLT,cash
Date,,,
2017-10-09,120.264023,111.911880,1
2017-10-10,120.548782,112.092407,1
2017-10-11,120.714127,112.309074,1
2017-10-12,120.567146,112.787476,1
2017-10-13,120.658981,113.599854,1
...,...,...,...
2022-09-29,181.940002,103.542809,1
2022-09-30,179.470001,102.206001,1
2022-10-03,184.029999,103.830002,1


Symbols,VTI,TLT,cash
Date,,,
2017-10-09,0.6,0.4,0.0
2017-10-10,0.6,0.4,0.0
2017-10-11,0.6,0.4,0.0
2017-10-12,0.6,0.4,0.0
2017-10-13,0.6,0.4,0.0
...,...,...,...
2022-09-29,0.6,0.4,0.0
2022-09-30,0.6,0.4,0.0
2022-10-03,0.6,0.4,0.0


Symbols,VTI,TLT,cash
Date,,,
2017-10-09,100,10,10000.0
2017-10-10,30,8,0.0
2017-10-11,0.0,0.0,0.0
2017-10-12,0.0,0.0,0.0
2017-10-13,0.0,0.0,0.0
...,...,...,...
2022-09-29,0.0,0.0,0.0
2022-09-30,0.0,0.0,0.0
2022-10-03,0.0,0.0,0.0


Symbols,VTI,TLT,cash
Date,,,
2017-10-09,100,10,10000.0
2017-10-10,30,8,0.0
2017-10-11,0.0,0.0,0.0
2017-10-12,0.0,0.0,0.0
2017-10-13,0.0,0.0,0.0
...,...,...,...
2022-09-29,0.0,0.0,0.0
2022-09-30,0.0,0.0,0.0
2022-10-03,0.0,0.0,0.0


In [163]:
n = len(price_df)

n = 2

result = pd.DataFrame()

for i in range(n):
    
    # Slice data to next time interval (t)
    t = pf_df.index[i]
    pf = pf_df.iloc[i]
    price = price_df.iloc[i]
    pnl = pnl_df.iloc[i]

    # Mark-to-market the portfolio 
    pnl = pf * price
    nav = pnl.sum()

    # Calculate latest portfolio weights based on NAV
    curr_weight = pnl / nav
     
    # Calculate delta of current to target weight
    target_weight = weight_df.iloc[i]
    delta_weight = target_weight - curr_weight

    # Based on buffer decide if trade should be made
    do_trade = delta_weight > trade_buffer

    # Assume min fixed commission so trade to target weight
    adj_target_weight = target_weight
    adj_target_value = adj_target_weight * nav
    adj_target_size = adj_target_value / price

    trade_value = pnl - adj_target_value
    trade_size = trade_value / price

    # Append data for this time interval to the result 
    series = pd.concat(
        [price, pf, pnl, curr_weight, target_weight, delta_weight, do_trade, adj_target_weight, trade_value, trade_size], 
        keys=['price', 'pf', 'pnl', 'curr_weight', 'target_weight', 'delta_weight', 'do_trade', 'adj_target_weight', 'trade_value', 'trade_size'],
        axis=1)
    series['datetime'] = t
    series = series.set_index(['datetime', series.index])
    result = pd.concat([result, series])


display(result)


price       pf           pnl curr_weight  \
datetime   Symbols                                                  
2017-10-09 VTI      120.264023      100  12026.402283      0.5196   
           TLT      111.911880       10   1119.118805    0.048351   
           cash       1.000000  10000.0       10000.0    0.432049   
2017-10-10 VTI      120.548782       30    3616.46347    0.801308   
           TLT      112.092407        8    896.739258    0.198692   
           cash       1.000000      0.0           0.0         0.0   

                    target_weight delta_weight  do_trade  trade_weight  \
datetime   Symbols                                                       
2017-10-09 VTI                0.6       0.0804      True           0.6   
           TLT                0.4     0.351649      True           0.4   
           cash               0.0    -0.432049     False           0.0   
2017-10-10 VTI                0.6    -0.201308     False           0.6   
           TLT                0.4     0.201308      True           0.4   
           cash               0.0          0.0     False           0.0   

                     trade_value  trade_size  
datetime   Symbols                            
2017-10-09 VTI      13887.312653  115.473542  
           TLT       9258.208435   82.727664  
           cash         0.000000    0.000000  
2017-10-10 VTI       2707.921637   22.463285  
           TLT       1805.281091   16.105293  
           cash         0.000000    0.000000